In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import RobustScaler

# Load the saved models
multiclass_model = load_model(r"C:\Users\LENOVO FLEX\Downloads\Intrusion Detection System/multiclass_model.h5")
binary_model = load_model(r"C:\Users\LENOVO FLEX\Downloads\Intrusion Detection System/binary_model.h5")

# Example input (replace with your actual input data)
input_data = {
    'sloss': 3,
    'rate': 34.050365,
    'dmean': 57,
    'sinpkt': 61.406365,
    'ct_dst_src_ltm': 2,
    'service': 'ftp',
    'ct_srv_src': 3,
    'smean': 218,
    'dbytes': 682,
    'ct_state_ttl': 1,
    'ct_srv_dst': 3,
    'sbytes': 2616,
    'proto': 'tcp',
    'ct_dst_sport_ltm': 1,
    'sttl': 254
}

# Convert input to DataFrame
input_df = pd.DataFrame([input_data])

# Ensure the columns are in the correct order
input_df = input_df[['sloss', 'rate', 'dmean', 'sinpkt', 'ct_dst_src_ltm', 'service',
                     'ct_srv_src', 'smean', 'dbytes', 'ct_state_ttl', 'ct_srv_dst', 'sbytes',
                     'proto', 'ct_dst_sport_ltm', 'sttl']]

# Scale the input data using the same RobustScaler
scaler = RobustScaler()
input_scaled = scaler.fit_transform(input_df)

# Reshape for multiclass LSTM model
input_reshaped = np.expand_dims(input_scaled, axis=1)

# Get predictions
# Multiclass Model Prediction
multiclass_prediction = multiclass_model.predict(input_reshaped)
multiclass_label = np.argmax(multiclass_prediction, axis=1)

# Binary Model Prediction
binary_prediction = binary_model.predict(input_scaled)
binary_label = (binary_prediction > 0.5).astype(int)

# Map numerical labels to UNSW attack types
attack_type_mapping = {
    0: "Normal",
    1: "Generic",
    2: "Exploits",
    3: "Fuzzers",
    4: "DoS",
    5: "Reconnaissance",
    6: "Analysis",
    7: "Backdoor",
    8: "Shellcode",
    9: "Worms"
}

# Decode the predicted label to the actual attack type
predicted_attack_type = attack_type_mapping[multiclass_label[0]]

# Print results
print("Multiclass Model Prediction:")
print(f"Predicted Class: {multiclass_label[0]}")
print(f"Predicted Attack Type: {predicted_attack_type}")
print(f"Class Probabilities: {multiclass_prediction}")

print("\nBinary Model Prediction:")
print(f"Predicted Label: {binary_label[0][0]} (0 = Normal, 1 = Attack)")
print(f"Probability: {binary_prediction[0][0]:.4f}")